In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
%cd '/content'
# this is saved in /content/drive/MyDrive/Colab Notebooks
# need to move it to Colab-Notebooks which is git checked in and make sure you
# are editing this copy


/content


# VLLM **Impl**

<ul>

<ul>
<li>Implement a llm served by vllm </li>
<li>wont work. on google drive; FUSE timeout</li>
<li></li>
</ul>

In [20]:
import os
os.getcwd()

'/content'

In [21]:
!git clone https://github.com/dougc333/vllm.git

Cloning into 'vllm'...
remote: Enumerating objects: 130292, done.
remote: Total 130292 (delta 0), reused 0 (delta 0), pack-reused 130292 (from 1)
Receiving objects: 100% (130292/130292), 111.63 MiB | 25.25 MiB/s, done.
Resolving deltas: 100% (102366/102366), done.


In [22]:
!/usr/bin/curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.9.20 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [23]:
!uv venv --python 3.10 --seed

Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment with seed packages at: .venv
 + pip==25.3
 + setuptools==80.9.0
 + wheel==0.45.1
Activate with: source .venv/bin/activate


In [24]:
!source .venv/bin/activate

In [25]:
!DEBIAN_FRONTEND=noninteractive \
     apt-get -o Dpkg::Options::=--force-confnew \
             -o Dpkg::Options::=--force-confdef \
             install -y gcc-12 g++-12 libnuma-dev python3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libnuma-dev is already the newest version (2.0.14-3ubuntu2).
gcc-12 is already the newest version (12.3.0-1ubuntu1~22.04.2).
python3-dev is already the newest version (3.10.6-1~22.04.1).
g++-12 is already the newest version (12.3.0-1ubuntu1~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [26]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12


In [27]:
%cd vllm

/content/vllm


In [28]:
#had to add --python 3.10 else it goes to 3.12
#cant run from cell, run from terminal
!uv pip install  -r requirements/build.txt --index-strategy unsafe-best-match

Using Python 3.12.12 environment at: /usr
Audited 10 packages in 95ms


In [29]:
!apt-get install -y ccache


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ccache is already the newest version (4.5.1-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [31]:
import os
os.environ["MAX_JOBS"] = "4"

Remove xformers{platform_machine == 'x86_64' and sys_platform
      == 'linux'}==0.0.33+5d4b92a5.
      
      
      d20251029
from requirements/cuda.txt


In [32]:
!uv pip install --python 3.10 -r  requirements/build.txt --index-strategy unsafe-best-match

Using Python 3.10.12 environment at: /usr
Audited 10 packages in 86ms


edit reqirements/cuda.txt comment out xformers{platform_machine == 'x86_64' and sys_platform == 'linux'}==0.0.33+5d4b92a5.

In [34]:
#cant run from cell, run from terminal window.
#eliminate xformers line
#had to add --python 3.10 else it goes to 3.12
#!grep -n "xformers" requirements/cuda.txt
#comment out: xformers from cuda.txt
!uv pip install --python 3.10 -r  requirements/cuda.txt --index-strategy unsafe-best-match

Using Python 3.10.12 environment at: /usr
Audited 57 packages in 97ms


In [35]:
# can leave off the --python 3.10, should work on 3.12
!VLLM_TARGET_DEVICE=cuda uv pip install --python 3.10 . -v --no-build-isolation -v

Streaming output truncated to the last 5000 lines.
      build/lib.linux-x86_64-cpython-310/vllm/model_executor/layers/quantization/quark/schemes
      copying
      vllm/model_executor/layers/quantization/quark/schemes/quark_w8a8_fp8.py ->
      build/lib.linux-x86_64-cpython-310/vllm/model_executor/layers/quantization/quark/schemes
      copying
      vllm/model_executor/layers/quantization/quark/schemes/__init__.py ->
      build/lib.linux-x86_64-cpython-310/vllm/model_executor/layers/quantization/quark/schemes
      copying
      vllm/model_executor/layers/quantization/quark/schemes/quark_ocp_mx.py ->
      build/lib.linux-x86_64-cpython-310/vllm/model_executor/layers/quantization/quark/schemes
      copying
      vllm/model_executor/layers/quantization/quark/schemes/quark_scheme.py ->
      build/lib.linux-x86_64-cpython-310/vllm/model_executor/layers/quantization/quark/schemes
      creating
      build/lib.linux-x86_64-cpython-310/vllm/model_executor/models/transformers
      co

In [ ]:
import vllm
vllm.__version__

In [ ]:
from vllm import LLM
llm = LLM("gpt2")

engine = llm.llm_engine
print(engine.model_executor.driver_worker.device)

In [ ]:
# ============================================================
# TinyLLM on WikiText-2 (decoder-only Transformer, from scratch)
# Colab-ready, pure PyTorch + HuggingFace Datasets/Tokenizer
# ============================================================

!pip install -q datasets transformers

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# -----------------------------
# 1. Hyperparameters
# -----------------------------
block_size   = 128      # max sequence length
batch_size   = 32
n_layers     = 4
n_heads      = 4
d_model      = 256
d_ff         = 4 * d_model
dropout      = 0.1
learning_rate = 3e-4
max_steps    = 2000      # small for demo; bump up to really train
print_every  = 100


# -----------------------------
# 2. Load WikiText-2 and tokenizer
# -----------------------------
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

tokenizer = AutoTokenizer.from_pretrained("gpt2")
# ensure we have a pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

vocab_size = tokenizer.vocab_size
print("Vocab size:", vocab_size)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# -----------------------------
# 3. Turn token stream into fixed-length blocks
# -----------------------------
def make_block_dataset(split):
    ids = tokenized[split]["input_ids"]
    # flatten all sequences into one long list
    all_ids = [tok for seq in ids for tok in seq]
    # trim so it divides evenly into block_size
    n = (len(all_ids) // block_size) * block_size
    all_ids = all_ids[:n]
    # shape: (num_blocks, block_size)
    data = torch.tensor(all_ids, dtype=torch.long).view(-1, block_size)
    # inputs: everything except last token; targets: everything except first token
    x = data[:, :-1]
    y = data[:, 1:]
    return x, y

train_x, train_y = make_block_dataset("train")
val_x,   val_y   = make_block_dataset("validation")

print("Train blocks:", train_x.shape, "Val blocks:", val_x.shape)

class BlockDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.size(0)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_ds = BlockDataset(train_x, train_y)
val_ds   = BlockDataset(val_x,   val_y)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, drop_last=True)


# -----------------------------
# 4. TinyLLM model (GPT-style)
# -----------------------------
class CausalSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, block_size, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads   = n_heads
        self.head_dim  = d_model // n_heads
        self.block_size = block_size

        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.attn_drop = nn.Dropout(dropout)
        self.resid_drop = nn.Dropout(dropout)

        # causal mask: (1, 1, T, T)
        mask = torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size)
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.qkv(x)  # (B, T, 3*C)
        q, k, v = qkv.chunk(3, dim=-1)

        # shape (B, n_heads, T, head_dim)
        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (B, h, T, T)
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)

        y = att @ v  # (B, h, T, head_dim)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_drop(self.proj(y))
        return y


class Block(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, block_size, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.attn = CausalSelfAttention(d_model, n_heads, block_size, dropout)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x


class TinyLLM(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, block_size, dropout=0.1):
        super().__init__()
        self.block_size = block_size
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(block_size,  d_model)
        self.drop = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([
            Block(d_model, n_heads, d_ff, block_size, dropout) for _ in range(n_layers)
        ])
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if isinstance(module, nn.Linear) and module.bias is not None:
            nn.init.zeros_(module.bias)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        assert T <= self.block_size, "Sequence too long"

        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)  # (1, T)
        x = self.tok_emb(idx) + self.pos_emb(pos)  # (B, T, C)
        x = self.drop(x)

        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)  # (B, T, vocab_size)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1),
                ignore_index=-100
            )
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        # simple autoregressive sampling
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float("Inf")

            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_token], dim=1)
        return idx

model = TinyLLM(
    vocab_size=vocab_size,
    d_model=d_model,
    n_layers=n_layers,
    n_heads=n_heads,
    d_ff=d_ff,
    block_size=block_size-1,  # because we use T-1 inputs
    dropout=dropout,
).to(device)

print("Number of parameters:", sum(p.numel() for p in model.parameters()) / 1e6, "M")


# -----------------------------
# 5. Training loop
# -----------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

def evaluate(loader, max_batches=50):
    model.eval()
    losses = []
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            if i >= max_batches:
                break
            x = x.to(device)
            y = y.to(device)
            _, loss = model(x, y)
            losses.append(loss.item())
    model.train()
    return sum(losses) / len(losses) if losses else None

step = 0
while step < max_steps:
    for x, y in train_loader:
        step += 1
        x = x.to(device)
        y = y.to(device)

        _, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        if step % print_every == 0:
            val_loss = evaluate(val_loader)
            print(f"Step {step}/{max_steps} | train loss: {loss.item():.4f} | val loss: {val_loss:.4f}")

        if step >= max_steps:
            break

print("Training finished.")


# -----------------------------
# 6. Quick generation demo
# -----------------------------
model.eval()
prompt = "Wikipedia is"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
out = model.generate(input_ids, max_new_tokens=50, temperature=0.8, top_k=50)
print("=== SAMPLE ===")
print(tokenizer.decode(out[0]))

Using device: cuda


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

...

Vocab size: 50257


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Train blocks: torch.Size([18686, 127]) Val blocks: torch.Size([1931, 127])
Number of parameters: 28.923648 M
Step 100/2000 | train loss: 6.9055 | val loss: 6.9795
Step 200/2000 | train loss: 6.6285 | val loss: 6.5550
Step 300/2000 | train loss: 6.4293 | val loss: 6.3343
Step 400/2000 | train loss: 6.0299 | val loss: 6.1891
Step 500/2000 | train loss: 6.2031 | val loss: 6.0776
Step 600/2000 | train loss: 5.9669 | val loss: 5.9886
Step 700/2000 | train loss: 5.7596 | val loss: 5.9239
Step 800/2000 | train loss: 5.5217 | val loss: 5.8452
Step 900/2000 | train loss: 5.6529 | val loss: 5.7835
Step 1000/2000 | train loss: 5.5582 | val loss: 5.7263
Step 1100/2000 | train loss: 5.4097 | val loss: 5.6738
Step 1200/2000 | train loss: 5.1860 | val loss: 5.6362
Step 1300/2000 | train loss: 5.1045 | val loss: 5.6093
Step 1400/2000 | train loss: 5.1163 | val loss: 5.5651
Step 1500/2000 | train loss: 5.0822 | val loss: 5.5286
Step 1600/2000 | train loss: 5.1152 | val loss: 5.4892
Step 1700/2000 | tra

In [ ]:
import os
save_dir = "tinyllm_wikitext2"

os.makedirs(save_dir, exist_ok=True)

# 1) Save model weights
torch.save(model.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))

# 2) Save tokenizer
tokenizer.save_pretrained(save_dir)

# 3) Save config / hyperparams
import json

config = {
    "vocab_size": vocab_size,
    "d_model": d_model,
    "n_layers": n_layers,
    "n_heads": n_heads,
    "d_ff": d_ff,
    "block_size": block_size - 1,
    "dropout": dropout,
}
with open(os.path.join(save_dir, "tinyllm_config.json"), "w") as f:
    json.dump(config, f, indent=2)

print(f"Saved TinyLLM + tokenizer to: {save_dir}")

Saved TinyLLM + tokenizer to: tinyllm_wikitext2


In [ ]:
import json
import torch
#
with open("tinyllm_wikitext2/tinyllm_config.json") as f:
    cfg = json.load(f)

reloaded_model = TinyLLM(
    vocab_size=cfg["vocab_size"],
    d_model=cfg["d_model"],
    n_layers=cfg["n_layers"],
    n_heads=cfg["n_heads"],
    d_ff=cfg["d_ff"],
    block_size=cfg["block_size"],
    dropout=cfg["dropout"],
).to(device)

state = torch.load("tinyllm_wikitext2/pytorch_model.bin", map_location=device)
reloaded_model.load_state_dict(state)
reloaded_model.eval()

tok = AutoTokenizer.from_pretrained("tinyllm_wikitext2")
prompt = "Wikipedia is"
inp = tok.encode(prompt, return_tensors="pt").to(device)
out = reloaded_model.generate(inp, max_new_tokens=50, temperature=0.8, top_k=50)
print(tok.decode(out[0]))

Wikipedia is the first season of the season . The first half @-@ month player , including the final game , one two games , respectively . 
 = = = = = Production = = = 
 = = = Track listing = = = = 


In [ ]:
# GPT-2 supported by vllm.

In [ ]:
from transformers import AutoConfig
cfg = AutoConfig.from_pretrained("/content/tinyllm_wikitext2")
print(cfg)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "TinyLLMForCausalLM"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 128,
  "n_embd": 256,
  "n_head": 4,
  "n_inner": null,
  "n_layer": 4,
  "n_positions": 128,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 50257
}



In [ ]:
import os, json
from transformers import GPT2Config

save_dir = "/content/tinyllm_wikitext2"

# Your current tiny config:
raw = {
  "vocab_size": 50257,
  "d_model": 256,
  "n_layers": 4,
  "n_heads": 4,
  "d_ff": 1024,
  "block_size": 127,
  "dropout": 0.1,
}

config = GPT2Config(
    vocab_size     = raw["vocab_size"],
    n_positions    = raw["block_size"] + 1,  # because your model uses T-1 inputs
    n_ctx          = raw["block_size"] + 1,
    n_embd         = raw["d_model"],
    n_layer        = raw["n_layers"],
    n_head         = raw["n_heads"],
    activation_function = "gelu_new",
    resid_pdrop    = raw["dropout"],
    embd_pdrop     = raw["dropout"],
    attn_pdrop     = raw["dropout"],
)

cfg = config.to_dict()
cfg["_name_or_path"] = "tinyllm-wikitext2"
cfg["architectures"] = ["TinyLLMForCausalLM"]  # <- vLLM will look for this backend

with open(os.path.join(save_dir, "config.json"), "w") as f:
    json.dump(cfg, f, indent=2)

print("Wrote new config.json")

Wrote new config.json


In [ ]:
from transformers import AutoConfig

cfg = AutoConfig.from_pretrained("/content/tinyllm_wikitext2")
print(type(cfg), cfg.model_type, cfg.n_embd, cfg.n_layer, cfg.n_head)

<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'> gpt2 256 4 4


In [ ]:
import os, json
from transformers import GPT2Config, AutoTokenizer

save_dir = "/content/tinyllm_wikitext2"

# use the tokenizer you trained with to get bos/eos/pad
tok = AutoTokenizer.from_pretrained("gpt2")
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

config = GPT2Config(
    vocab_size = tok.vocab_size,
    n_positions = 128,
    n_ctx       = 128,
    n_embd      = 256,
    n_layer     = 4,
    n_head      = 4,
    activation_function = "gelu_new",
    resid_pdrop = 0.1,
    embd_pdrop  = 0.1,
    attn_pdrop  = 0.1,
    bos_token_id = tok.bos_token_id or tok.eos_token_id,
    eos_token_id = tok.eos_token_id,
    pad_token_id = tok.pad_token_id,
)

cfg = config.to_dict()
cfg["_name_or_path"] = "tinyllm-wikitext2"
cfg["architectures"] = ["TinyLLMForCausalLM"]  # your vLLM backend
cfg["id2label"] = {"0": "LABEL_0"}            # if you really want this

with open(os.path.join(save_dir, "config.json"), "w") as f:
    json.dump(cfg, f, indent=2)

print("Wrote clean config.json")

Wrote clean config.json


In [ ]:
!pip install "vllm>=0.5.0" "transformers>=4.45.0" torch

In [ ]:
import os
import json
from dataclasses import dataclass
from typing import Optional, Tuple, Union

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    PretrainedConfig,
    PreTrainedModel,
    AutoTokenizer,
)
from transformers.modeling_outputs import CausalLMOutputWithPast

# In recent HF, ALL_ATTENTION_FUNCTIONS is shared between backends.
# If this import fails, search in your transformers installation for ALL_ATTENTION_FUNCTIONS
from transformers.models.llama.modeling_llama import ALL_ATTENTION_FUNCTIONS

from vllm import LLM


# -----------------------------
# 1. Config
# -----------------------------
class ToyConfig(PretrainedConfig):
    model_type = "toy-transformer"

    def __init__(
        self,
        vocab_size: int = 50257,   # match GPT-2 tokenizer
        hidden_size: int = 128,
        num_hidden_layers: int = 2,
        num_attention_heads: int = 4,
        intermediate_size: int = 256,
        max_position_embeddings: int = 512,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = intermediate_size
        self.max_position_embeddings = max_position_embeddings

        # vLLM/Transformers backend stuff
        # This tells vLLM that the base model supports its attention backend.
        self._attn_implementation = "vllm"
        self.is_decoder = True
        self.is_encoder_decoder = False


# -----------------------------
# 2. Attention block compatible with vLLM backend
# -----------------------------
class ToyAttention(nn.Module):
    # decoder-only causal LM, so attention is causal
    is_causal = True

    def __init__(self, config: ToyConfig):
        super().__init__()
        self.config = config
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads

        self.q_proj = nn.Linear(config.hidden_size, config.hidden_size)
        self.k_proj = nn.Linear(config.hidden_size, config.hidden_size)
        self.v_proj = nn.Linear(config.hidden_size, config.hidden_size)
        self.o_proj = nn.Linear(config.hidden_size, config.hidden_size)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        bsz, seqlen, h = hidden_states.shape
        head_dim = self.head_dim
        num_heads = self.num_heads

        # project to Q, K, V
        q = self.q_proj(hidden_states)
        k = self.k_proj(hidden_states)
        v = self.v_proj(hidden_states)

        # (bsz, seqlen, num_heads, head_dim) -> (bsz, num_heads, seqlen, head_dim)
        def shape(x):
            return (
                x.view(bsz, seqlen, num_heads, head_dim)
                .transpose(1, 2)
                .contiguous()
            )

        q = shape(q)
        k = shape(k)
        v = shape(v)

        # This is the crucial hook: use vLLM attention backend
        attention_interface = ALL_ATTENTION_FUNCTIONS[self.config._attn_implementation]
        attn_output, attn_weights = attention_interface(
            self,
            query_states=q,
            key_states=k,
            value_states=v,
            attention_mask=attention_mask,
            is_causal=self.is_causal,
            **kwargs,
        )

        # (bsz, num_heads, seqlen, head_dim) -> (bsz, seqlen, h)
        attn_output = (
            attn_output.transpose(1, 2)
            .reshape(bsz, seqlen, h)
            .contiguous()
        )
        attn_output = self.o_proj(attn_output)
        return attn_output, attn_weights


class ToyMLP(nn.Module):
    def __init__(self, config: ToyConfig):
        super().__init__()
        self.fc1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.fc2 = nn.Linear(config.intermediate_size, config.hidden_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.fc2(F.gelu(self.fc1(x)))


class ToyBlock(nn.Module):
    def __init__(self, config: ToyConfig):
        super().__init__()
        self.self_attn = ToyAttention(config)
        self.mlp = ToyMLP(config)
        self.ln1 = nn.LayerNorm(config.hidden_size)
        self.ln2 = nn.LayerNorm(config.hidden_size)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> torch.Tensor:
        # Self-attention
        residual = hidden_states
        hidden_states = self.ln1(hidden_states)
        attn_output, _ = self.self_attn(
            hidden_states,
            attention_mask=attention_mask,
            **kwargs,
        )
        hidden_states = residual + attn_output

        # MLP
        residual = hidden_states
        hidden_states = self.ln2(hidden_states)
        hidden_states = residual + self.mlp(hidden_states)
        return hidden_states


# -----------------------------
# 3. Base model (decoder LM) + LM head
# -----------------------------
class ToyModel(PreTrainedModel):
    config_class = ToyConfig
    _supports_attention_backend = True  # required for vLLM backend

    def __init__(self, config: ToyConfig):
        super().__init__(config)
        self.config = config

        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.embed_positions = nn.Embedding(
            config.max_position_embeddings, config.hidden_size
        )

        self.layers = nn.ModuleList(
            [ToyBlock(config) for _ in range(config.num_hidden_layers)]
        )
        self.ln_f = nn.LayerNorm(config.hidden_size)

        # Important for generation: LM head
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        self.post_init()

    def get_input_embeddings(self):
        return self.embed_tokens

    def set_input_embeddings(self, new_embeddings):
        self.embed_tokens = new_embeddings

    def get_output_embeddings(self):
        return self.lm_head

    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings

    def forward(
        self,
        input_ids: torch.LongTensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        **kwargs,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        bsz, seqlen = input_ids.shape

        if position_ids is None:
            position_ids = torch.arange(
                seqlen, dtype=torch.long, device=input_ids.device
            )
            position_ids = position_ids.unsqueeze(0).expand(bsz, seqlen)

        inputs_embeds = self.embed_tokens(input_ids)
        pos_embeds = self.embed_positions(position_ids)
        hidden_states = inputs_embeds + pos_embeds

        # (very simple): just apply full attention over all tokens
        for layer in self.layers:
            hidden_states = layer(
                hidden_states,
                attention_mask=attention_mask,
                **kwargs,
            )

        hidden_states = self.ln_f(hidden_states)
        logits = self.lm_head(hidden_states)

        loss = None
        if labels is not None:
            # standard LM shift
            shift_logits = logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, self.config.vocab_size),
                shift_labels.view(-1),
            )

        if not self.config.use_return_dict:
            output = (logits,)
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=None,
            hidden_states=None,
            attentions=None,
        )


# -----------------------------
# 4. Save as a HF-style model directory
# -----------------------------
def save_toy_model(model_dir: str = "toy_vllm_model"):
    os.makedirs(model_dir, exist_ok=True)

    # use GPT-2 tokenizer
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.save_pretrained(model_dir)

    config = ToyConfig(
        vocab_size=tokenizer.vocab_size,
        hidden_size=128,
        num_hidden_layers=2,
        num_attention_heads=4,
        intermediate_size=256,
        max_position_embeddings=512,
    )

    # Add HF-style auto_map so Transformers/vLLM know how to import this class
    auto_map = {
        "AutoConfig": "__main__.ToyConfig",
        # For vLLM + Transformers backend, the base model should be the one in auto_map.
        # We treat ToyModel as the causal LM base.
        "AutoModelForCausalLM": "__main__.ToyModel",
    }

    cfg_dict = config.to_dict()
    cfg_dict["auto_map"] = auto_map
    cfg_dict["architectures"] = ["ToyModel"]

    with open(os.path.join(model_dir, "config.json"), "w") as f:
        json.dump(cfg_dict, f, indent=2)

    model = ToyModel(config)
    model.save_pretrained(model_dir)

    print(f"Saved toy model + tokenizer to: {model_dir}")


# -----------------------------
# 5. Load with vLLM and generate
# -----------------------------
def run_with_vllm(model_dir: str = "toy_vllm_model"):
    # Force Transformers backend; we’re using a Transformers-style custom model
    llm = LLM(model=model_dir, model_impl="transformers", trust_remote_code=True)

    prompts = ["Hello world", "Once upon a time"]
    outputs = llm.generate(prompts, max_tokens=20)

    for i, out in enumerate(outputs):
        print(f"\nPrompt: {prompts[i]!r}")
        print("Completion:", out.outputs[0].text)


if __name__ == "__main__":
    save_toy_model()
    run_with_vllm()

In [ ]:
# training toymodel
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from toy_vllm_model import ToyModel, ToyConfig
from datasets import load_dataset

# Load config + model
config = ToyConfig()
model = ToyModel(config)

# Load a text dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, max_length=128)
tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])

# Data collator for next-token prediction
collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,        # causal LM objective (autoregressive)
)

args = TrainingArguments(
    output_dir="toy_trained",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-4,
    warmup_steps=100,
    num_train_epochs=1,
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="epoch",
    fp16=False,          # you can enable fp16 if you want
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator=collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"]
)

trainer.train()

In [ ]:
# full example w training

from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from toy_vllm_model import ToyModel, ToyConfig, save_toy_model
from datasets import load_dataset

# Step 1: save base toy model
save_toy_model("toy_vllm_model")

# Step 2: load base model for training
tokenizer = AutoTokenizer.from_pretrained("gpt2")
config = ToyConfig(vocab_size=tokenizer.vocab_size)
model = ToyModel(config)

# Step 3: load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenized = dataset.map(
    lambda e: tokenizer(e["text"], truncation=True, max_length=128),
    batched=True,
    remove_columns=["text"],
)

# Step 4: collator
collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Step 5: training args
args = TrainingArguments(
    output_dir="toy_vllm_trained",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    learning_rate=3e-4,
    warmup_steps=50,
    num_train_epochs=1,
    save_strategy="epoch",
    logging_steps=25,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=collator,
)

trainer.train()

# Step 6: save trained model in HF format
trainer.save_model("toy_vllm_trained")

print("Training complete! Now you can serve it in vLLM:")
print("vllm serve toy_vllm_trained --model-impl transformers --trust-remote-code")

In [ ]:
vllm serve toy_vllm_trained --model-impl transformers --trust-remote-code